<a href="https://colab.research.google.com/github/mouktik05/research/blob/main/mouktik_cdna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install biopython requests


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 7.9 MB/s eta 0:00:00


In [2]:
import requests

In [3]:
def get_transcript_exons(transcript_id):
    """Fetch exon information for a given Ensembl transcript ID."""
    #url = f"https://rest.ensembl.org/overlap/id/{transcript_id}?feature=gene;content-type=application/json"
    url = f"https://grch37.rest.ensembl.org/overlap/id/{transcript_id}?feature=gene;content-type=application/json"
    #https://grch37.rest.ensembl.org/overlap/id/ENST00000025008?feature=gene;content-type=application/json
    #https://rest.ensembl.org/overlap/id/ENST00000025008?feature=gene;content-type=application/json
    #response = requests.get(url, headers={"Content-Type": "application/json"})
    response = requests.get(url, json={"start": "value"})
    if response.status_code != 200:
        raise Exception(f"API request failed with status code {response.status_code}")
    return response.json()

def map_cdna_to_genomic(transcript_id, cdna_position):
    """Map a cDNA position to a genomic coordinate using Ensembl."""
    exons = get_transcript_exons(transcript_id)
    total_cdna_len = 0
    for exon in sorted(exons, key=lambda x: x['start']):
        exon_cdna_start = total_cdna_len + 1
        exon_cdna_end = total_cdna_len + exon['end'] - exon['start'] + 1
        total_cdna_len = exon_cdna_end

        if exon_cdna_start <= cdna_position <= exon_cdna_end:
            genomic_pos = exon['start'] + (cdna_position - exon_cdna_start)
            return exon['seq_region_name'], genomic_pos, exon['strand']

    raise ValueError(f"cDNA position {cdna_position} out of range for transcript {transcript_id}")


In [4]:

transcript_id = "ENST00000008527"  # Replace with your transcript ID for USP28
cdna_position = 1589  # Replace with your cDNA position
chromosome, genomic_position, strand = map_cdna_to_genomic(transcript_id, cdna_position)
print(f"Chromosome: {chromosome}, Genomic Position: {genomic_position}, Strand: {strand}")



Chromosome: 12, Genomic Position: 107386730, Strand: -1
